In [4]:
import random
import math

from graph import Graph

random.seed(0)

In [6]:
def create_graph_from_txt_file(txt_file: str):
    nodes = []
    with open(txt_file) as file:
        line = file.readline()
        while line:
            if "#" not in line:
                n_info = line.replace("\n", "").split(",")
                nodes.append((n_info[0], n_info[1], n_info[2]))

            line = file.readline()

    if len(nodes) > 0:
        g = Graph()
        g.add_node_list(nodes)
        return g

    raise Exception("Wrong input file provided")

In [ ]:
class SimulateAnnealing:
    def __init__(self, 
                 graph: Graph, 
                 initial_temperature: float, 
                 cooling_rate: float,
                 max_iterations: int,
                 initial_solution: list = None):
        self.graph = graph

        self.current_solution = initial_solution
        self.best_solution = initial_solution

        self.temperature = initial_temperature
        self.cooling_rate = cooling_rate
        self.max_iterations = max_iterations

    
    def random_initial_solution(self):
        '''
        Generate a random initial solution
        '''
        if self.current_solution is None:
            nodes = self.graph.node_count()

            curr_node = random.choice(nodes)
            self.current_solution = [curr_node]

            available_nodes = set(nodes).remove(curr_node)
            while available_nodes:
                next_node = min(available_nodes, key=lambda x: self.graph.get_edge_weight(curr_node, x))
                available_nodes.remove(next_node)
                self.current_solution.append(next_node)
                curr_node = next_node

            self.best_solution = self.current_solution


    def cost(self, solution: list):
        '''
        Calculate the cost of the solution
        '''
        cost = sum(self.graph.get_edge_weight(u, v) for u, v in zip(solution, solution[1:] + [solution[0]]))
    
        return cost
    
    def generate_new_solution(self):
        '''
        Generate a new solution by reversing a random subsequence of the current solution
        '''
        nodes = self.graph.node_count()

        neighbour = list(self.current_solution)
        l = random.randint(2, nodes-1)
        i = random.randint(0, nodes-l)
        neighbour[i : (i + l)] = reversed(neighbour[i : (i + l)])

        return neighbour
    
    def acceptance_test(self, candidate_solution: list):
        '''
        Acceptance test for the new solution
        '''
        candidate_cost = self.cost(candidate_solution)
        current_cost = self.cost(self.current_solution)
        if candidate_cost < current_cost:
            self.current_solution = candidate_solution
            if candidate_cost < self.cost(self.best_solution):
                self.best_solution = candidate_solution
        else:
            if random.random() < math.exp((current_cost - candidate_cost) / self.temperature):
                self.current_solution = candidate_solution

    def update_temperature(self):
        self.temperature *= self.cooling_rate

    def run(self):
        # generate initial solution
        self.random_initial_solution()

        curr_iter = 0
        while curr_iter < self.max_iterations:
            # generate new solution
            candidate_solution = self.generate_new_solution()

            # evaluate new solution
            self.acceptance_test(candidate_solution)

            # decrease temperature
            self.update_temperature()

            curr_iter += 1

        return self.best_solution, self.cost(self.best_solution)